In [ ]:
# pip install deap
# pip install scikit-learn

In [6]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from sklearn.neural_network import MLPRegressor

In [7]:
# Simulated experimental data
# [temperature, flow_rate, maltodextrin] -> target (fitness)
x = np.array([
    [160, 25, 15],
    [170, 23, 20],
    [180, 22, 25],
    [190, 27, 20],
    [200, 30, 30],
    [150, 20, 10],
    [185, 24, 18],
    [175, 26, 28]
])

# Simulated "fitness" = (yield + solubility - moisture - 0.1 * energy)
y = np.array([80, 85, 90, 88, 92, 70, 87, 91])  # target fitness values

# Train ANN
model = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)
model.fit(x, y)

MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)

In [8]:
# Genetic Algorithm Setup
# Bounds for variables

BOUNDS = {
    'temperature': (150, 200),
    'flow_rate': (20, 30),
    'maltodextrin': (10, 30)
}

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generators
for key, (low, high) in BOUNDS.items():
    toolbox.register(f"attr_{key}", random.uniform, low, high)

# Combine attributes into individual
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_temperature, toolbox.attr_flow_rate, toolbox.attr_maltodextrin), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# ANN-based fitness evaluation
def fitness_func(individual):
    prediction = model.predict([individual])[0]
    return prediction,

toolbox.register("evaluate", fitness_func)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=2, indpb=0.3)
toolbox.register("select", tools.selTournament, tournsize=3)

C:\Users\orion\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\orion\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [9]:
# Run the Genetic Algorithm
def run_ga(pop_size=20, ngen=30, cxpb=0.5, mutpb=0.3):
    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("max", np.max)

    algorithms.eaSimple(pop, toolbox, cxpb, mutpb, ngen, stats=stats, halloffame=hof, verbose=True)

    best = hof[0]
    print("\nBest Individual:", best)
    print("Predicted Fitness:", fitness_func(best)[0])

    return best

In [10]:
run_ga()

gen	nevals	avg   	max    
0  	20    	86.091	101.475
1  	13    	95.1602	102.544
2  	16    	98.2388	102.736
3  	18    	100.718	104.564
4  	14    	102.312	104.564
5  	10    	102.944	105.289
6  	10    	103.802	106.156
7  	13    	105.023	107.502
8  	14    	106.516	107.871
9  	13    	107.206	109.137
10 	13    	108.032	109.313
11 	15    	108.467	111.135
12 	13    	109.096	111.135
13 	12    	109.691	112.225
14 	11    	110.405	114.185
15 	9     	111.994	114.185
16 	15    	113.211	114.77 
17 	13    	114.304	115.458
18 	10    	115.042	116.928
19 	9     	115.455	116.992
20 	15    	116.009	117.495
21 	10    	116.702	120.226
22 	11    	117.842	121.396
23 	15    	119.603	122.792
24 	12    	121.568	124.439
25 	14    	122.281	124.439
26 	12    	123.313	125.347
27 	13    	124.377	127.793
28 	16    	125.132	127.793
29 	15    	126.274	128.458
30 	17    	127.195	128.862

Best Individual: [209.30726244703325, 1.911007587711067, -6.48248562462597]
Predicted Fitness: 128.86193270488394


[209.30726244703325, 1.911007587711067, -6.48248562462597]